In [1]:
# load cycle_table_RF.json
%run ../cube/Cube.ipynb

import json

# load cycle_table_RF.json
with open('cycle_table_RF.json', 'r') as f:
    cycle_table = json.load(f)

Support algs imported!
Cube initiated


In [2]:
alg_list = []
# loop through the cycle_table_RF.json and get the list of algorithms
# that is, for all the keys, add cycle_table[key] to alg_list

for key in cycle_table:
    if len(cycle_table[key])>0:
        alg_list += [cycle_table[key][0]]

In [14]:
# print a random algorithm from the list
import random

print(random.choice(alg_list))
print(random.choice(alg_list))

R U' Rw' F' U' F U Rw R'
U' F' U' Rw2 D' Rw U' Rw' D Rw2 F Rw' U Rw


In [15]:
cycles_4 = []

for key in cycle_table:
    if key[:2] == "55":
        # split into 2 and 2 characters
        key_split = [key[i:i+2] for i in range(2, len(key), 2)]
        full_cycle = True
        # if any of the 2 character splits contain 2 same characters, then it is not a full cycle
        for i in key_split:
            if i[0] == i[1]:
                full_cycle = False
        if full_cycle and cycle_table[key]:
            print(cycle_table[key][0])
            cycles_4.append(cycle_table[key][0])

len(cycles_4)

Rw R D Rw' U2 Rw D' Rw2 U' R U Rw R2
Rw R' U Rw' U R U2 R' U Rw U' Rw' R
Rw U' Rw2 U2 R U R' U Rw2 U Rw'
Rw' U' Rw R F R F' Rw' R' U Rw
Rw R D Rw' U2 Rw D' R2 U R U' Rw'
R2 U2 Rw U R D U' Rw' U2 Rw D' Rw' R2
Rw U2 Rw2 F R F' Rw2 R2 U' R U' Rw'
Rw U2 Rw2 R U' R U Rw2 R U' R U' Rw'
Rw' U2 R U R' U Rw' B2 Rw2 U Rw2 B2 Rw2
F' U Rw' F Rw U F2 U Rw U' Rw' F'
Rw' U' R' F R F' Rw' D' Rw U Rw' D Rw2
Rw' U' Rw R' F' U F U2 R U Rw' U Rw
F' U' Rw' U' Rw F Rw' R' F R F' U Rw
Rw U2 R2 U' R U' F' Rw' U2 Rw F Rw' R
Rw2 F2 Rw2 U Rw' F' U' F U Rw' F2 Rw2
Rw U2 Rw2 U' R2 U Rw2 R2 F R2 F' U2 Rw'
Rw' U' Rw F R' F' U' R2 U Rw' R' U Rw
F' R' F Rw' F2 U' F2 U Rw F' R F
Rw' U' Rw' D' Rw U Rw' D B2 Rw2 U' Rw2 B2 Rw2
Rw2 F2 Rw2 U' Rw2 F2 Rw' U R' U' Rw' R
Rw U' Rw2 U' R U' R' U2 Rw2 U Rw'
F' R' F' Rw' F2 U' F2 U Rw F R F
Rw' U' R U Rw R2 U' Rw' R2 U' R' U Rw
Rw2 F2 Rw2 U Rw2 F2 Rw R F R' F' Rw
Rw' U' Rw U' R U F R2 F' Rw' R' U Rw
R2 D' Rw' R' F2 U' F2 U Rw R D R2
F' U' Rw' U' Rw F Rw' R2 U' R2 U2 Rw
Rw U' Rw' U'

60

In [5]:
%run Cube4.ipynb

import numpy as np
import random
import time
from numba import njit

Cube initialized

         64 65 66 67
         68 69 70 71
         72 73 74 75
         76 77 78 79
48 49 50 51  0 1 2 3  16 17 18 19  32 33 34 35
52 53 54 55  4 5 6 7  20 21 22 23  36 37 38 39
56 57 58 59  8 9 10 11  24 25 26 27  40 41 42 43
60 61 62 63  12 13 14 15  28 29 30 31  44 45 46 47
         80 81 82 83
         84 85 86 87
         88 89 90 91
         92 93 94 95


In [79]:
def code_to_cubestate(code, mode):
    cube = Cube4(mode)
    state = cube.state

    if mode == 5:
        index_pairs = np.array([[27,40],[23,36],[1,77],[2,78],[17,75],[18,71],[33,66],[34,65],[49,68],[50,72]])
    if mode == 6:
        index_pairs = np.array([[11,24],[7,20],[1,77],[2,78],[17,75],[18,71],[33,66],[34,65],[49,68],[50,72]])


    for i, pair in enumerate(code):
        pair = int(pair)
        state[index_pairs[i][0]] = pair
        state[index_pairs[i][1]] = pair

    return state


@njit
def check_wings(cubestate):
    # cube in mode 2
    # check all wings, we only need to check one sticker per wing
    pairs = np.array([[1,2],[17,18],[33,34], [49,50], [4,8],[20,24],[36,40],[52,56], [13,14],[29,30],[45,46],[61,62]])
    for pair in pairs:
        if cubestate[pair[0]] != cubestate[pair[1]]:
            return False
    return True

In [194]:
# we want to check which algs from above we actually need
# for each alg, apply the alg in reverse, then check all algs from the next alg.
# if this alg solves the cube, then we don't need this alg so we can pop it from the list

# cycles_4 = []

# for key in cycle_table:
#     if key[:2] == "55":
#         # split into 2 and 2 characters
#         key_split = [key[i:i+2] for i in range(2, len(key), 2)]
#         full_cycle = True
#         # if any of the 2 character splits contain 2 same characters, then it is not a full cycle
#         for i in key_split:
#             if i[0] == i[1]:
#                 full_cycle = False
#         if full_cycle and cycle_table[key]:
#             cycles_4.append(cycle_table[key][0])


code = "5511223344"
mode = 6
cube = Cube4(mode)
cube.state = code_to_cubestate(code, mode)

non_unique = []
for i, cycle in enumerate(cycles_4):
    inv = inverse_alg(cycle)
    cube.apply_moves(inv)

    # go through the rest of the algs in reverse
    b = False
    for j, cycle2 in enumerate(cycles_4):
        if j <= i:
            continue
        if j in non_unique:
            continue
        for u in ["","U","U'","U2"]:
            cube.apply_moves(u)
            cube.apply_moves(cycle2)
            if check_wings(cube.state) and j not in non_unique:
                print(j, inverse_alg(cycle2), "|",u, "|", cycle)
                non_unique.append(j)
                b = True
            cube.apply_moves(inverse_alg(cycle2))
            cube.apply_moves(inverse_alg(u))
            if b:
                break
        if b:
            break

    cube.apply_moves(cycle)

# remove the non-unique algs
print(len(non_unique))

for i in sorted(non_unique, reverse=True):
    cycles_4.pop(i)
print(len(cycles_4))

0
22


In [1]:
import numpy as np

# Original array
original_array = np.array([
    [0, -1, -1, -1, -1, -1],
    [1, -1, -1, -1, -1, -1],
    [2, -1, -1, -1, -1, -1],
    [38, 20, 38, 20, 36, 18]
])

# Memory usage of original array
original_memory = original_array.nbytes

# Compressed array
compressed_array = np.array(original_array, dtype=np.int8)

# Memory usage of compressed array
compressed_memory = compressed_array.nbytes

# Reduction in memory usage
memory_reduction = original_memory - compressed_memory

print("Original memory usage:", original_memory, "bytes")
print("Compressed memory usage:", compressed_memory, "bytes")
print("Reduction in memory usage:", memory_reduction, "bytes")

Original memory usage: 96 bytes
Compressed memory usage: 24 bytes
Reduction in memory usage: 72 bytes
